In [1]:
import pandas as pd
from pathlib import Path

In [2]:
interim = Path('../data/interim')
panel = pd.read_pickle(interim / 'panel.pkl')
ratings538 = pd.read_pickle(Path(interim) / 'ratings538.pkl')
teams = pd.read_pickle(Path(interim) / 'teams.pkl')

In [3]:
ratings_teams = ratings538.reset_index().groupby(['TEAM_ID', 'year']).first().reset_index()[['TEAM_ID', 'year']]

In [4]:
recode = {
    'MIA': 'FLA',
    'NYG': 'SFG',
    'BRO': 'LAD',
    'MON': 'WSN',
    'SLB': 'BAL',
    'TBR': 'TBD',
    'WSA': 'TEX',
    'SEP': 'MIL',
    'CAL': 'ANA',
    'LAA': 'ANA',
    'WSH': 'MIN',
    'KCA': 'OAK',
    'PHA': 'OAK',
    'BSN': 'ATL',
    'MLN': 'ATL',
}
for old_code, new_code in recode.items():
    teams.loc[teams.teamIDBR == old_code, 'teamIDBR'] = new_code

In [5]:
teams2021 = teams.loc[teams.year == 2020, :]
teams2021.loc[:, ['year']] = 2021
teams = teams.append(teams2021)

/Users/rsfletch/opt/anaconda3/envs/bts/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [6]:
team_map = pd.merge(
    ratings_teams, 
    teams.loc[:, ['year', 'teamIDretro', 'teamIDBR']],  
    left_on = ['year', 'TEAM_ID'],
    right_on = ['year', 'teamIDBR'],
)

In [7]:
new = pd.merge(
    ratings538.reset_index(), 
    team_map[['TEAM_ID', 'teamIDretro']], 
    on=['TEAM_ID'], 
    how='left', 
)

del new['TEAM_ID']
new = new.rename(columns={'teamIDretro': 'TEAM_ID'})
new = new.set_index(['GAME_ID', 'TEAM_ID'])

In [8]:
new

year  Home      elo_pre  elo_prob   rating_pre  \
GAME_ID      TEAM_ID                                                   
CHW192004140 CHA      1920     1  1533.645000  0.551343  1532.576000   
             CHA      1920     1  1533.645000  0.551343  1532.576000   
             CHA      1920     1  1533.645000  0.551343  1532.576000   
             CHA      1920     1  1533.645000  0.551343  1532.576000   
             CHA      1920     1  1533.645000  0.551343  1532.576000   
...                    ...   ...          ...       ...          ...   
WSN202110030 BOS      2021     0  1500.122331  0.435368  1504.356267   
             BOS      2021     0  1500.122331  0.435368  1504.356267   
             BOS      2021     0  1500.122331  0.435368  1504.356267   
             BOS      2021     0  1500.122331  0.435368  1504.356267   
             BOS      2021     0  1500.122331  0.435368  1504.356267   

                      rating_prob  pitcher_rgs  pitcher_adj  
GAME_ID      TEAM_ID                                         
CHW192004140 CHA         0.552751       51.701          NaN  
             CHA         0.552751       51.701          NaN  
             CHA         0.552751       51.701          NaN  
             CHA         0.552751       51.701          NaN  
             CHA         0.552751       51.701          NaN  
...                           ...          ...          ...  
WSN202110030 BOS         0.442932          NaN          NaN  
             BOS         0.442932          NaN          NaN  
             BOS         0.442932          NaN          NaN  
             BOS         0.442932          NaN          NaN  
             BOS         0.442932          NaN          NaN  

[31509844 rows x 8 columns]